In [82]:
from yargy.tokenizer import MorphTokenizer
from nltk.corpus import stopwords
import pandas as pd
import re

df = pd.read_json('data/cintra_phoenix_oils_hr_mgck_feather.json').set_index('id')

tokenizer = MorphTokenizer()
ru_stopwords = set(stopwords.words("russian"))

df = df['quote'].apply(lambda x: re.findall(r'\b\w+\b', str(x).lower())).to_frame()
df = df['quote'].apply(lambda x: ' '.join(x)).to_frame()

list_tok_2 = [[] for _ in range(len(df.quote.values))]
for i in range(len(df.quote.values)):
    list_tok = []
    tok = tokenizer(df.quote[i])
    for t in tok:
        if t.normalized not in ru_stopwords:
            list_tok.append(t.normalized)
    list_tok_2[i] = ' '.join(list_tok)
df['quote'] = list_tok_2

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer

vectorizer = TfidfVectorizer(max_df=0.2, min_df=15)
# vectorizer = TfidfVectorizer()
# vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform([text for text in df.quote.values]).toarray()

In [95]:
from sklearn.decomposition import LatentDirichletAllocation

num_topics = 5
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

for topic_idx, topic_words in enumerate(lda.components_):
    top_words_idx = topic_words.argsort()[-5:][::-1]
    top_words = [vectorizer.get_feature_names_out()[i] for i in top_words_idx]
    print(f"Тема {topic_idx + 1}: {', '.join(top_words)}")

Тема 1: хороший, конкретно, условие, поход, снежный
Тема 2: цк, цмф, просто, друг, тип
Тема 3: человек, знать, смена, очень, говорить
Тема 4: сказать, змс, квартал, премия, мёд
Тема 5: дон, понимать, старый, феникс, приходиться


___

In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Подготовка данных
documents = df.quote.values
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents)

# Кластерный анализ (KMeans)
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# Вывод примеров документов в каждом кластере
for cluster_idx in range(num_clusters):
    cluster_documents = [documents[i] for i, label in enumerate(kmeans.labels_) if label == cluster_idx]
    print(cluster_documents)
    # print(f"Кластер {cluster_idx + 1}:")
    # for doc in cluster_documents:
    #     print(f" - {doc[0:150]}")

['собеседование сказать нужно мыть пол помыть пол следить порядок уборщица сказать руководитель девочка который сказать нужно мыть туалет это сказать скажем', 'сказать цмф начало давать премия итог квартал сказать это просто взрыв вроде всё стать хороший работать сказать красный дракон стать слух ходить условие хороший сказать', 'живой общение свой слово говорить просто немного это стандарт это сказать это мочь понятно нибыть предложить свободный казаться чувствовать сказать это всё балл', 'пруберцо сказать всё праздник сказать проводить разный сказать бывало отдел ма ркетинг сказать заказать торт сказать просто собраться столовъть сказать цмф сказать всё день рождение отмечать сказать это праздник сказать вся компания сначала казаться сказать приятно сказать ру ководство сказать заботиться сказать сказать постоянный торт сказать застолье сказать стать раздражать сказать всё это сказать пооидти сказать слишком далеко сказать всё равно сказать работа сказать праздник сказать сказать фон

In [110]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


___

In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
 
nltk.download('wordnet')
 
# создаём объект этого класса
lemmatizer = WordNetLemmatizer()
 
# и пустой список для слов после лемматизации
lemmatized = []
 
# проходимся по всем токенам
for token in df.quote.values:
 
    # применяем лемматизацию
    token = lemmatizer.lemmatize(token)
 
    # добавляем слово после лемматизации в список
    lemmatized.append(token)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\orenc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['ннас среди ноч i райоон 55 час упасть пол x вода сттеклять тара тобыть посреди лужа сстеккнуть всёё это поод утро кстати ээтый перрвыя падать неправилььный рассчитывать должный выставлять определённый картинка пара чтоо грохаться полка буттылка', 'программа повышение квалификация гильдия цмф хитрый хитрый весь хотеть повышаться свой предпочтение направление гильдия гильдия толкать свой нужно сотрудник белый змея например программа гибкий сотрудник выбирать нужно', 'мурра новиград город контраст цмф змс человек превратиться кот всё занятый никто хотеть протянуть лапка товарищ помочь груз всё напряжение спешка мурра ранний хороший работать втроём порознь', 'тип полгода ждать установка фильтр маслобаза думать бесконечный огненный огонь затягивать поставка давно всё порядок', 'всьё просто ранний бумажка летать теряться искать приходиться весь зеленос всё место плантир e весь инф o рм a ция ру k ой ничо пропасть a ют работать стать гораздо удобный', 'расказыыывалий лия вва м 0 жет рууково

In [87]:
print(lemmatized[1])

программа повышение квалификация гильдия цмф хитрый хитрый весь хотеть повышаться свой предпочтение направление гильдия гильдия толкать свой нужно сотрудник белый змея например программа гибкий сотрудник выбирать нужно


___

In [29]:
tokenizer = MorphTokenizer()
ru_stopwords = set(stopwords.words("russian"))

In [30]:
df = df['quote'].apply(lambda x: re.findall(r'\b\w+\b', str(x).lower())).to_frame()
df = df['quote'].apply(lambda x: ' '.join(x)).to_frame()

In [31]:
list_tok_2 = [[] for _ in range(len(df.quote.values))]
for i in range(len(df.quote.values)):
    list_tok = []
    tok = tokenizer(df.quote[i])
    for t in tok:
        if t.normalized not in ru_stopwords:
            list_tok.append(t.normalized)
    list_tok_2[i] = ' '.join(list_tok)

df['quote'] = list_tok_2

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer

# vectorizer = TfidfVectorizer(max_df=0.2, min_df=10)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([text for text in df.quote.values]).toarray()

In [334]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
X = pca.fit_transform(X)

In [335]:
from sklearn.cluster import KMeans

k_means = KMeans(n_clusters=10, n_init="auto")
k_means.fit(X)

KMeans(n_clusters=10)

In [336]:
result = k_means.predict(X)

In [337]:
# from collections import Counter

# cluster_to_stat = [Counter() for _ in range(10)]

# for cluster, doc in zip(result, df.id):
#     cluster_to_stat[cluster].update([doc])

# cluster_to_stat[2]

In [339]:
df = df.reset_index()

from sklearn.svm import SVC

y = [text for text in df.id.values]

svc = SVC()
svc.fit(X, y)

SVC()

In [358]:
text = 'проблема с мышами каждый вечер мыши достали'
tok = tokenizer(text)
list_tok = []
for t in tok:
        if t.normalized not in ru_stopwords:
            list_tok.append(t.normalized)
text = ' '.join(list_tok)
vectorizer = TfidfVectorizer()

vect = vectorizer.fit_transform([text]).toarray()

pca = PCA(n_components=50)
vect = pca.fit_transform(vect)

# vect = pca.transform(vect)
vect
svc.predict(vect)

ValueError: n_components=50 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'

___

In [32]:
df.head()

,quote
id,
0,ннас среди ноч i райоон 55 час упасть пол x во...
1,программа повышение квалификация гильдия цмф х...
2,мурра новиград город контраст цмф змс человек ...
3,тип полгода ждать установка фильтр маслобаза д...
4,всьё просто ранний бумажка летать теряться иск...


In [36]:
X.shape

(959, 3)

In [15]:
from gensim import corpora
from gensim.models import LdaModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

# Подготовка данных
documents = list(df.quote.values)
stop_words = set(STOPWORDS)
texts = [[word for word in simple_preprocess(doc) if word not in stop_words] for doc in documents]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Обучение модели LDA
num_topics = 15
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Вывод тем и связанных с ними слов
for idx, topic in lda_model.print_topics(-1):
    print(f"Тема {idx + 1}: {topic}")

Тема 1: 0.036*"как" + 0.026*"не" + 0.021*"то" + 0.021*"что" + 0.019*"бы" + 0.018*"на" + 0.016*"так" + 0.012*"но" + 0.010*"вот" + 0.008*"это"
Тема 2: 0.020*"это" + 0.016*"что" + 0.015*"на" + 0.013*"как" + 0.011*"но" + 0.010*"не" + 0.008*"все" + 0.007*"понимаешь" + 0.007*"просто" + 0.006*"змс"
Тема 3: 0.023*"понимаешь" + 0.013*"но" + 0.011*"это" + 0.010*"не" + 0.009*"что" + 0.008*"всё" + 0.008*"по" + 0.008*"на" + 0.007*"да" + 0.007*"как"
Тема 4: 0.023*"что" + 0.019*"не" + 0.018*"это" + 0.017*"то" + 0.011*"по" + 0.010*"всё" + 0.009*"на" + 0.008*"есть" + 0.008*"мы" + 0.007*"как"
Тема 5: 0.047*"как" + 0.043*"бы" + 0.008*"цмф" + 0.007*"все" + 0.007*"на" + 0.007*"но" + 0.006*"для" + 0.006*"ну" + 0.005*"это" + 0.005*"не"
Тема 6: 0.016*"как" + 0.011*"всё" + 0.009*"на" + 0.007*"что" + 0.007*"не" + 0.006*"так" + 0.005*"когда" + 0.005*"понимаешь" + 0.004*"просто" + 0.004*"это"
Тема 7: 0.024*"как" + 0.018*"не" + 0.016*"что" + 0.013*"на" + 0.010*"это" + 0.008*"все" + 0.008*"то" + 0.007*"но" + 0.006*

In [55]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Подготовка данных
documents = ["Текст статьи 1", "Текст статьи 2", "Текст статьи 3"]
vectorizer = CountVectorizer(max_features=1000, stop_words='english')
X = vectorizer.fit_transform(documents)

# Применение LDA
num_topics = 5
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# Вывод тем и связанных с ними слов
for topic_idx, topic_words in enumerate(lda.components_):
    top_words_idx = topic_words.argsort()[-10:][::-1]
    top_words = [vectorizer.get_feature_names()[i] for i in top_words_idx]
    print(f"Тема {topic_idx + 1}: {', '.join(top_words)}")

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names'

____

In [ ]:
import gzip

from dataclasses import dataclass
from typing import Iterator

from nltk.corpus import stopwords
from yargy.tokenizer import MorphTokenizer


@dataclass
class Text:
    label: str
    title: str
    text: str


def read_texts(fn: str) -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))


tokenizer = MorphTokenizer()
ru_stopwords = set(stopwords.words("russian"))


def normalize_text(text: str) -> str:
    tokens = [
        tok.normalized for tok in tokenizer(text) if tok.normalized not in ru_stopwords
    ]
    return " ".join(tokens)

In [ ]:
normalize_text("Привет, миру!")

In [ ]:
texts = list(read_texts("data/news.txt.gz"))

vectorizer = TfidfVectorizer(max_df=0.2, min_df=10)
# vectorizer = HashingVectorizer()

X = vectorizer.fit_transform([normalize_text(text.text) for text in texts]).toarray()

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=500)
X = pca.fit_transform(X)

In [ ]:
def transform_text(text: str) -> np.ndarray:
    normalized_text = normalize_text(text)
    vect = vectorizer.transform([normalized_text]).toarray()
    return pca.transform(vect)

transform_text("привет миру ещё раз")

In [ ]:
from sklearn.cluster import KMeans

k_means = KMeans(n_clusters=10, n_init="auto")
k_means.fit(X)

In [ ]:
from sklearn.svm import SVC

y = [text.label for text in texts]

svc = SVC()
svc.fit(X, y)

In [ ]:
svc.predict(transform_text("путешествие в Японию"))